In [1]:
import pandas as pd # for working with dataframes
import os
from os import listdir, makedirs # for retrieving files from directory
from os.path import isfile, join # for retrieving files from directory
from pathlib import Path # for retrieving files from directory
import networkx as nx # for making graphs
import numpy as np # for arrays
import matplotlib.pyplot as plt # for plotting
import matplotlib.image as mpimg # for images
from scipy.interpolate import interp1d # for interpolating points
from scipy.spatial import procrustes # for Procrustes analysis
from sklearn.decomposition import PCA # for principal component analysis
import seaborn as sns# ; sns.set(color_codes=True) # for plotting in seaborn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # for LDA
from sklearn.metrics import confusion_matrix # for confusion matrix
from sklearn.model_selection import StratifiedKFold # for Stratified K fold sampling
from sklearn.cluster import AgglomerativeClustering # for agglomerative clustering
import scipy.stats as stats # for kruskal wallis test
import statsmodels.stats.multitest as multitest # multiple test adjustment
from numpy.linalg import det # for sampling higher dimensional convex hull
from scipy.stats import dirichlet
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
import math
from shapely.geometry import Polygon, Point
from shapely.ops import nearest_points
import trimesh
import pickle

In [22]:
#################
### FUNCTIONS ###
#################

def angle_between(p1, p2, p3):
    """
    define a function to find the angle between 3 points anti-clockwise in degrees, p2 being the vertex
    inputs: three angle points, as tuples
    output: angle in degrees
    """
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    deg1 = (360 + math.degrees(math.atan2(x1 - x2, y1 - y2))) % 360
    deg2 = (360 + math.degrees(math.atan2(x3 - x2, y3 - y2))) % 360
    return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)

def rotate_points(xvals, yvals, degrees):
    """"
    define a function to rotate 2D x and y coordinate points around the origin
    inputs: x and y vals (can take pandas dataframe columns) and the degrees (positive, anticlockwise) to rotate
    outputs: rotated and y vals
    """
    angle_to_move = 90-degrees
    rads = np.deg2rad(angle_to_move)
    
    new_xvals = xvals*np.cos(rads)-yvals*np.sin(rads)
    new_yvals = xvals*np.sin(rads)+yvals*np.cos(rads)
    
    return new_xvals, new_yvals

def interpolation(x, y, number): 
    """
    define a function to return equally spaced, interpolated points for a given polyline
    inputs: arrays of x and y values for a polyline, number of points to interpolate
    ouputs: interpolated points along the polyline, inclusive of start and end points
    """
    distance = np.cumsum(np.sqrt( np.ediff1d(x, to_begin=0)**2 + np.ediff1d(y, to_begin=0)**2 ))
    distance = distance/distance[-1]

    fx, fy = interp1d( distance, x ), interp1d( distance, y )

    alpha = np.linspace(0, 1, number)
    x_regular, y_regular = fx(alpha), fy(alpha)
    
    return x_regular, y_regular

def euclid_dist(x1, y1, x2, y2):
    """
    define a function to return the euclidean distance between two points
    inputs: x and y values of the two points
    output: the eulidean distance
    """
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def poly_area(x,y):
    """
    define a function to calculate the area of a polygon using the shoelace algorithm
    inputs: separate numpy arrays of x and y coordinate values
    outputs: the area of the polygon
    """
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def gpa_mean(leaf_arr, landmark_num, dim_num):
    
    """
    define a function that given an array of landmark data returns the Generalized Procrustes Analysis mean
    inputs: a 3 dimensional array of samples by landmarks by coordinate values, number of landmarks, number of dimensions
    output: an array of the Generalized Procrustes Analysis mean shape
    
    """

    ref_ind = 0 # select arbitrary reference index to calculate procrustes distances to
    ref_shape = leaf_arr[ref_ind, :, :] # select the reference shape

    mean_diff = 10**(-30) # set a distance between means to stop the algorithm

    old_mean = ref_shape # for the first comparison between means, set old_mean to an arbitrary reference shape

    d = 1000000 # set d initially arbitraily high

    while d > mean_diff: # set boolean criterion for Procrustes distance between mean to stop calculations

        arr = np.zeros( ((len(leaf_arr)),landmark_num,dim_num) ) # empty 3D array: # samples, landmarks, coord vals

        for i in range(len(leaf_arr)): # for each leaf shape 

            s1, s2, distance = procrustes(old_mean, leaf_arr[i]) # calculate procrustes adjusted shape to ref for current leaf
            arr[i] = s2 # store procrustes adjusted shape to array

        new_mean = np.mean(arr, axis=(0)) # calculate mean of all shapes adjusted to reference

        s1, s2, d = procrustes(old_mean, new_mean) # calculate procrustes distance of new mean to old mean

        old_mean = new_mean # set the old_mean to the new_mea before beginning another iteration

    return new_mean

# From: https://stackoverflow.com/questions/59073952/how-to-get-uniformly-distributed-points-in-convex-hull
# Accessed 11 February 2024

def dist_in_hull(points, n):
    dims = points.shape[-1]
    hull = points[ConvexHull(points).vertices]
    deln = hull[Delaunay(hull).simplices]

    vols = np.abs(det(deln[:, :dims, :] - deln[:, dims:, :])) / math.factorial(dims)    
    sample = np.random.choice(len(vols), size = n, p = vols / vols.sum())

    return np.einsum('ijk, ij -> ik', deln[sample], dirichlet.rvs([1]*(dims + 1), size = n))

def is_self_intersecting(polygon_coordinates):
    """
    Checks if a polygon is self-intersecting.

    Args:
        polygon_coordinates: A list of (x, y) tuples representing the polygon's vertices.

    Returns:
        True if the polygon is self-intersecting, False otherwise.
    """
    polygon = Polygon(polygon_coordinates)
    return not polygon.is_simple

def grid_points_in_polygon(polygon_coords, spacing):
    """
    Generates a grid of points within a polygon.

    Args:
        polygon_coords: A list of (x, y) tuples defining the polygon's vertices.
        spacing: The desired spacing between grid points.

    Returns:
        A list of (x, y) tuples representing the grid points within the polygon.
    """
    polygon = Polygon(polygon_coords)
    min_x, min_y, max_x, max_y = polygon.bounds

    x_coords = np.arange(min_x, max_x + spacing, spacing)
    y_coords = np.arange(min_y, max_y + spacing, spacing)
    xv, yv = np.meshgrid(x_coords, y_coords)

    grid_points_x = []
    grid_points_y = []
    for x, y in zip(xv.flatten(), yv.flatten()):
        point = Point(x, y)
        if polygon.contains(point):
            grid_points_x.append(x)
            grid_points_y.append(y)

    grid_points = np.column_stack((grid_points_x, grid_points_y))

    return grid_points

def delaunay_triangulation_within_polygon(points, polygon):
    """
    Generates a Delaunay triangulation of a set of points that lie within a given polygon.

    Args:
        points: A list of (x, y) coordinate tuples or a NumPy array of shape (n, 2) representing the points.
        polygon: A {Link: Shapely Polygon https://shapely.readthedocs.io/en/stable/manual.html#polygon} object representing the polygon.

    Returns:
        A tuple containing:
        - A NumPy array of shape (n, 2) representing the coordinates of the vertices of the triangulation.
        - A NumPy array of shape (m, 3) representing the indices of the vertices that form each triangle.
    """

    # Create Delaunay triangulation
    delaunay = Delaunay(points)

    # Filter triangles that fall outside the polygon
    triangles_within_polygon = []
    for triangle_indices in delaunay.simplices:
        triangle_points = points[triangle_indices]
        triangle_polygon = Polygon(triangle_points)

        if triangle_polygon.within(polygon):
            triangles_within_polygon.append(triangle_indices)

    # Return the filtered triangles
    return points, np.array(triangles_within_polygon)

def shortest_distance_to_polygon(point_coords, polygon_coords):
    """
    Calculates the shortest distance from a point to a polygon outline.

    Args:
        point_coords (tuple): Coordinates of the point (x, y).
        polygon_coords (list): List of coordinate tuples defining the polygon's vertices.

    Returns:
        float: The shortest distance from the point to the polygon outline.
    """
    point = Point(point_coords)
    polygon = Polygon(polygon_coords)

    # Get the boundary (outline) of the polygon as a LinearRing
    polygon_boundary = polygon.boundary

    # Find the nearest point on the polygon boundary to the given point
    nearest_point = nearest_points(point, polygon_boundary)[1]

    # Calculate the distance between the point and the nearest point on the boundary
    distance = point.distance(nearest_point)

    return distance


def lift_points_radially(points_3d, angle_degrees):
    """
    Rotates each point around the axis perpendicular to its (x, y) vector,
    lifting it into the Z direction by the given angle.
    Points at the origin remain unchanged.

    Parameters:
        points_3d (np.ndarray): Nx3 array of points (x, y, z)
        angle_degrees (float): Angle in degrees to lift into Z

    Returns:
        np.ndarray: Nx3 array of rotated 3D points
    """
    angle_radians = np.deg2rad(angle_degrees)
    lifted_points = []

    for x, y, z in points_3d:
        if np.isclose(x, 0) and np.isclose(y, 0):
            # Leave origin fixed
            lifted_points.append([0, 0, z])
            continue

        # Get direction vector in XY plane from origin to point
        v_xy = np.array([x, y, 0])
        norm_xy = np.linalg.norm(v_xy)
        v_xy_unit = v_xy / norm_xy

        # Axis of rotation is perpendicular to (x, y, 0) in the XY plane
        # Cross with Z to get rotation axis
        axis = np.cross(v_xy_unit, [0, 0, 1])  # This is the rotation axis
        axis = axis / np.linalg.norm(axis)

        # Rodrigues' rotation formula
        v = np.array([x, y, z])
        k = axis
        cos_theta = np.cos(angle_radians)
        sin_theta = np.sin(angle_radians)

        v_rot = (v * cos_theta +
                 np.cross(k, v) * sin_theta +
                 k * np.dot(k, v) * (1 - cos_theta))

        lifted_points.append(v_rot)

    return np.array(lifted_points)

def triangle_area_2d(p0, p1, p2):
    return 0.5 * np.abs((p1[0] - p0[0]) * (p2[1] - p0[1]) -
                        (p2[0] - p0[0]) * (p1[1] - p0[1]))

def triangle_area_3d(p0, p1, p2):
    v1 = p1 - p0
    v2 = p2 - p0
    return 0.5 * np.linalg.norm(np.cross(v1, v2))

def preserve_area(vertices_2d, faces, z_raw):
    vertices_3d = np.hstack([vertices_2d, z_raw[:, np.newaxis]])
    z_adjusted = z_raw.copy()

    for tri in faces:
        i0, i1, i2 = tri
        p0, p1, p2 = vertices_2d[[i0, i1, i2]]
        a_2d = triangle_area_2d(p0, p1, p2)

        p0_3d = np.array([*p0, z_adjusted[i0]])
        p1_3d = np.array([*p1, z_adjusted[i1]])
        p2_3d = np.array([*p2, z_adjusted[i2]])
        a_3d = triangle_area_3d(p0_3d, p1_3d, p2_3d)

        if a_3d == 0 or a_2d == 0:
            continue  # skip degenerate triangles

        scale = np.sqrt(a_2d / a_3d)

        # Scale the Z-values locally to match the area
        centroid_z = (z_adjusted[i0] + z_adjusted[i1] + z_adjusted[i2]) / 3.0
        for i in [i0, i1, i2]:
            deviation = z_adjusted[i] - centroid_z
            z_adjusted[i] = centroid_z + deviation * scale

    return z_adjusted

def get_parameters(parameter_file_name):

    # read in parameter file
    parameters = pd.read_csv(parameter_file_name)

    # get parameter name
    parameter_name = parameters["parameter_name"].item()
    
    # morphospace and PC values to synthesize leaf shape
    morphospace_file = parameters["morphospace_file"].item()
    PC_val_file = parameters["PC_val_file"].item()
    
    # select two PC axes to sythesize leaves from
    PCa = parameters["PCa"].item() # index position of first PC
    PCb = parameters["PCb"].item() # index position of second PC
    
    # set density of grid to sample points within leaves
    grid_density = parameters["grid_density"].item() # density to sample points on leaves
    
    # select the range of leaf numbers to uniformly sample from
    low_lf_num = parameters["low_lf_num"].item() # inclusive
    high_lf_num = parameters["high_lf_num"].item() # up to, not including
    
    # set relative height of first leaf in series
    low_juv_len = parameters["low_juv_len"].item() # inclusive
    high_juv_len = parameters["high_juv_len"].item() # not including
    
    # set height of last leaf relative to one
    low_adult_len = parameters["low_adult_len"].item() # inclusive
    high_adult_len = parameters["high_adult_len"].item() # not including
    
    # set maximum scaling values for leaf curvature and leaf curling
    lf_curve = parameters["lf_curve"].item() # max scaling of leaf curvature (the interior of the leaf)
    lf_curl = parameters["lf_curl"].item() # max scaling of leaf curl (across length of leaf)
    
    # set maximum displacement at leaf tip (y value)
    max_down_curl = parameters["max_down_curl"].item()
    max_up_curl = parameters["max_up_curl"].item()
    
    # set minimum and maximum limits of leaf angles
    ang_min_limit = parameters["ang_min_limit"].item() # max possible min angle
    ang_max_limit = parameters["ang_max_limit"].item() # max possible angle
    
    # set phyllotaxy angle
    phyllo_ang = parameters["phyllo_ang"].item()

    return parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang

def generate_model_vals(parameter_file_name):
    
    # read in parameter values using get_parameters function
    parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang = get_parameters(parameter_file_name)

    # 0 = [-137.5077640500378546463487,137.5077640500378546463487]
    # 1 = [-137.5077640500378546463487,137.5077640500378546463487,90,180]
    # 2 = [-137.5077640500378546463487,137.5077640500378546463487,90,180,<random -360 to +360>]
    # 3 = [<random -360 to +360>]

    if phyllo_ang==0:
        phyllo_ang=np.random.choice(np.array([-137.5077640500378546463487,137.5077640500378546463487]))
    elif phyllo_ang==1:
        phyllo_ang=np.random.choice(np.array([-137.5077640500378546463487,137.5077640500378546463487,90,180]))
    elif phyllo_ang==2:
        phyllo_ang=np.random.choice(np.array([-137.5077640500378546463487,137.5077640500378546463487,90,180,np.random.uniform(-360,360)]))
    else:
        phyllo_ang=np.random.uniform(-360,360)
        
    # select the number of leaves in the series
    n = np.random.randint(low_lf_num, high_lf_num) 
    
    # select relative height of first leaf in series
    juv_len = np.random.uniform(low_juv_len, high_juv_len) 
    
    # set node position of max leaf height of 1 (0 to 1)
    node_max = np.random.uniform(0, 1) 
    
    # set the relative 
    adult_len = np.random.uniform(low_adult_len, high_adult_len) 
    
    # calculate the scaling factor for leaf curvature and leaf curl
    leaf_curve = np.random.uniform(0,lf_curve) 
    leaf_curl = np.random.uniform(0,lf_curl) 
    
    # calculate leaf curl amount
    leaf_tip = np.random.uniform(max_down_curl,max_up_curl) 
    
    # set middle of leaf curl (y value)
    leaf_mid = np.random.uniform(0,leaf_tip) 
    
    # calculate elevation angles
    angle_min = np.random.uniform(0,ang_min_limit) # get the minimum angle
    angle_max = np.random.uniform(angle_min,ang_max_limit) # get the maximum angle
    
    # find the two points in PCA space to make the leaf series
    # Load PCA model
    with open(morphospace_file, 'rb') as file:
        pca = pickle.load(file)
    # Load in PC values
    PCs = np.load(PC_val_file)
    
    # retrieve just the desired two PCs
    just_PCs = PCs[:,[PCa,PCb]]
    # create the convex hull
    hull = ConvexHull(just_PCs)
    # retreieve hull points
    hull_points = just_PCs[hull.vertices]
    # find two random points in 2D convex hull
    # these are the terminal points of the line
    term_pts = dist_in_hull(hull_points,2)
    # these are the points
    start_x = term_pts[0][0]
    start_y = term_pts[0][1]
    end_x = term_pts[1][0]
    end_y = term_pts[1][0]

    return parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y,end_x,end_y

def get_model_val_df(parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y,end_x,end_y):
    
    # create pandas dataframe to save files
    model_val_file = pd.DataFrame({
    "parameter_name":[parameter_name],
    "morphospace_file":[morphospace_file],
    "PC_val_file":[PC_val_file],
    "PCa":[PCa], 
    "PCb":[PCb], 
    "grid_density":[grid_density], 
    "low_lf_num":[low_lf_num], 
    "high_lf_num":[high_lf_num], 
    "low_juv_len":[low_juv_len], 
    "high_juv_len":[high_juv_len], 
    "low_adult_len":[low_adult_len], 
    "high_adult_len":[high_adult_len], 
    "lf_curve":[lf_curve], 
    "lf_curl":[lf_curl], 
    "max_down_curl":[max_down_curl], 
    "max_up_curl":[max_up_curl], 
    "ang_min_limit":[ang_min_limit], 
    "ang_max_limit":[ang_max_limit], 
    "phyllo_ang":[phyllo_ang],
    "n":[n],
    "juv_len":[juv_len],
    "node_max":[node_max],
    "adult_len":[adult_len],
    "leaf_curve":[leaf_curve],
    "leaf_curl":[leaf_curl],
    "leaf_tip":[leaf_tip],
    "leaf_mid":[leaf_mid],
    "angle_min":[angle_min],
    "angle_max":[angle_max],
    "start_x":[start_x], 
    "start_y":[start_y], 
    "end_x":[end_x],
    "end_y":[end_y],
    })

    # return the model value df
    return model_val_file

def get_model_vals(model_val_file_name):

    # read in parameter file
    model_vals = pd.read_csv(model_val_file_name)

    # get values
    parameter_name=model_vals["parameter_name"].item()
    morphospace_file=model_vals["morphospace_file"].item() 
    PC_val_file=model_vals["PC_val_file"].item() 
    PCa=model_vals["PCa"].item() 
    PCb=model_vals["PCb"].item() 
    grid_density=model_vals["grid_density"].item() 
    low_lf_num=model_vals["low_lf_num"].item() 
    high_lf_num=model_vals["high_lf_num"].item() 
    low_juv_len=model_vals["low_juv_len"].item() 
    high_juv_len=model_vals["high_juv_len"].item() 
    low_adult_len=model_vals["low_adult_len"].item() 
    high_adult_len=model_vals["high_adult_len"].item() 
    lf_curve=model_vals["lf_curve"].item() 
    lf_curl=model_vals["lf_curl"].item() 
    max_down_curl=model_vals["max_down_curl"].item() 
    max_up_curl=model_vals["max_up_curl"].item()
    ang_min_limit=model_vals["ang_min_limit"].item() 
    ang_max_limit=model_vals["ang_max_limit"].item() 
    phyllo_ang=model_vals["phyllo_ang"].item()
    n=model_vals["n"].item()
    juv_len=model_vals["juv_len"].item()
    node_max=model_vals["node_max"].item()
    adult_len=model_vals["adult_len"].item()
    leaf_curve=model_vals["leaf_curve"].item()
    leaf_curl=model_vals["leaf_curl"].item()
    leaf_tip=model_vals["leaf_tip"].item()
    leaf_mid=model_vals["leaf_mid"].item()
    angle_min=model_vals["angle_min"].item()
    angle_max=model_vals["angle_max"].item()
    start_x=model_vals["start_x"].item()
    start_y=model_vals["start_y"].item()
    end_x=model_vals["end_x"].item()
    end_y=model_vals["end_y"].item()
    
    return parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y, end_x,end_y

def fibonacci_sphere(samples=1000):
    points = []
    phi = math.pi * (math.sqrt(5.) - 1.)  # golden angle in radians
    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2  # y goes from 1 to -1
        radius = math.sqrt(1 - y * y)  # radius at y
        theta = phi * i  # golden angle increment
        x = math.cos(theta) * radius
        z = math.sin(theta) * radius
        points.append((x, y, z))
    return points

def ray_casting(n_origins,radius,n_sample,v,parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y, end_x,end_y):

    #####
    # READ BACK IN PCA INFORMATION
    #####
    pca = pickle.load(open(morphospace_file,'rb')) 
    PCs = np.load(PC_val_file)
    
    #####
    # SPECIFY NUMBER OF LEAVES IN LEAF SERIES
    #####
    
    # Create an array of evenly spaced values from 0 to 1
    t_values = np.linspace(0, 1, n)
    
    start_point = np.array([start_x, start_y])
    end_point = np.array([end_x, end_y])
    
    # Calculate the points using linear interpolation
    points = np.array([start_point + t * (end_point - start_point) for t in t_values])
    
    # calculate eigen leaves
    eigen_arr = np.zeros((len(points),np.shape(PCs)[1])) # create an array for eigenleaf, length of number of PCs
    eigen_arr[:,PCa] = points[:,0] # set the PCa values
    eigen_arr[:,PCb] = points[:,1] # set the PCb values
    
    # calculate the inverse eigenleaf
    inv_leaf = pca.inverse_transform(eigen_arr)
    inv_x = inv_leaf[:,0::2] # select just inverse x vals
    inv_y = inv_leaf[:,1::2] # select just inverse y vals
    
    # create an array of the leaf series
    leaf_lines = np.stack((inv_x, inv_y), axis=1)
    leaf_lines = np.swapaxes(leaf_lines, 1,2)
    
    #####
    # CHECK FOR SELF-INTERSECTION
    #####
    
    self_intersect = 0 # assume no self intersect
    for i in range(len(leaf_lines)):
        if is_self_intersecting(leaf_lines[i]):
            self_intersect+=1
        else:
            continue

    #####
    # TRANSLATE LEAF BASE TO ORIGIN
    #####
    
    trans_lfs = np.zeros(np.shape(leaf_lines))
    
    for i in range(len(leaf_lines)):
        start_pt = leaf_lines[i,0,:] # leaf start
        end_pt = leaf_lines[i,-1,:] # leaf end
        base_pt = np.mean((start_pt, end_pt), axis=0) # base point of leaf
        trans_lf = leaf_lines[i,:,:] - base_pt # translated lf
        trans_lfs[i,:,:] = trans_lf # save translated lf
    
    #####
    # SCALE LEAF LENGTHS ACROSS LEAF SERIES
    #####
    
    # find relative lengths for each leaf (to 1)
    coeffs = np.polyfit(x=[0,node_max,1], y=[juv_len,1,adult_len], deg=2) # calculate coefficients
    func = np.poly1d(coeffs) # calculate 2D polynomial function
    lf_lens = func(np.linspace(0,1,n)) # calculate leaf lengths relative to 1
    
    scaled_lfs = np.zeros(np.shape(trans_lfs))
    for i in range(n): # for each leaf in the series
        scale_len_lf = trans_lfs[i,:,:]/np.max(trans_lfs[i,:,1]) # scale leaf len to 1
        scale_len_lf = scale_len_lf*lf_lens[i] # scale leaf to desired length
        scaled_lfs[i,:,:] = scale_len_lf # save scaled length leaf
    
    
    #####
    # CREATE A LEAF SERIES FOR VALIDATION, ALL WITH V NUMBER LEAVES IN SERIES
    #####
    
    v_values = np.linspace(0, 1, v) # v is for validating leaf shapes
    v_points = np.array([start_point + v * (end_point - start_point) for v in v_values])
    valid_arr = np.zeros((len(v_points),np.shape(PCs)[1]))
    valid_arr[:,PCa] = v_points[:,0] # set the PCa values
    valid_arr[:,PCb] = v_points[:,1] # set the PCb values
    
    val_leaf = pca.inverse_transform(valid_arr)
    val_x = val_leaf[:,0::2] 
    val_y = val_leaf[:,1::2] 
    
    val_lines = np.stack((val_x, val_y), axis=1)
    val_lines = np.swapaxes(val_lines, 1,2)
    
    vtrans_lfs = np.zeros(np.shape(val_lines))
    for i in range(len(val_lines)):
        start_pt = val_lines[i,0,:] # leaf start
        end_pt = val_lines[i,-1,:] # leaf end
        base_pt = np.mean((start_pt, end_pt), axis=0) # base point of leaf
        vtrans_lf = val_lines[i,:,:] - base_pt # translated lf
        vtrans_lfs[i,:,:] = vtrans_lf # save translated lf
    
    # find relative lengths for each leaf (to 1)
    coeffs = np.polyfit(x=[0,node_max,1], y=[juv_len,1,adult_len], deg=2) # calculate coefficients
    func = np.poly1d(coeffs) # calculate 2D polynomial function
    vlf_lens = func(np.linspace(0,1,v)) # calculate leaf lengths relative to 1
    
    vscaled_lfs = np.zeros(np.shape(vtrans_lfs))
    for i in range(v): # for each leaf in the series
        vscale_len_lf = vtrans_lfs[i,:,:]/np.max(vtrans_lfs[i,:,1]) # scale leaf len to 1
        vscale_len_lf = vscale_len_lf*vlf_lens[i] # scale leaf to desired length
        vscaled_lfs[i,:,:] = vscale_len_lf # save scaled length leaf
    
    #####
    # CALCULATE LEAF SURFACE VALUES AND ELEVATION
    #####
    
    pts_3D = [] # list of x, y, z 
    tris_3D = [] # list of triangles
    
    for i in range(len(scaled_lfs)): # for each leaf
    
        curr_lf = scaled_lfs[i] # get current leaf
        
        grid_pts = grid_points_in_polygon(curr_lf, grid_density) # calculate grid points
        
        # Delauney triangulation
        # tris = indices of individual triangles in points
        pts, tris = delaunay_triangulation_within_polygon(grid_pts, Polygon(curr_lf)) # calculate Delauney triangulation
        
        # get distances of each grid point to polygon boundary
        dists=[]
        for j in range(len(pts)):
            dists.append(shortest_distance_to_polygon((pts[j,0],pts[j,1]),curr_lf))
        scaled_lf_curve = np.array(dists)*leaf_curve # get scaled leaf curvature
        
        # calculate leaf curl from distance to base
        coeffs = np.polyfit(x=[0,0.5,1], y=[0,leaf_mid,leaf_tip], deg=2) # calculate coefficients
        func = np.poly1d(coeffs) # calculate 2D polynomial function
        
        lf_curl_vals = func(pts[:,1])*leaf_curl*np.max(pts[:,1]) # calculate leaf lengths relative to 1
        
        lf_surf_vals = scaled_lf_curve+lf_curl_vals # calculate overall curvature
    
        lf_surf_lf = np.column_stack((pts, lf_surf_vals)) # combine x, y, and z vals
    
        # translate the z axis to the origin
        lf_surf_lf[:,2] = lf_surf_lf[:,2] - np.mean(lf_surf_lf[:,2][lf_surf_lf[:,1]==np.min(lf_surf_lf[:,1])]  )
    
        # rotate leaf
        rot_x, rot_y = rotate_points(lf_surf_lf[:,0],lf_surf_lf[:,1], phyllo_ang*i)
        lf_surf_lf[:,0:2] = np.column_stack((rot_x, rot_y))
    
        # Warp Dlauney triangulation into 3D
        # get surface values of leaf
        z_raw = lf_surf_lf[:,2]
    
        # get the x and y coordinate values
        vertices_2d = lf_surf_lf[:,0:2]
        
        # Area-preserving adjustment
        z_warped = preserve_area(vertices_2d, tris, z_raw)
        
        # Now vertices_3d = [x, y, z_warped]
        vertices_3d = np.hstack([vertices_2d, z_warped[:, np.newaxis]])
    
        # Elevate the leaves
        # elevate the points
        angs = np.linspace(angle_min, angle_max, n)
        elv_pts = lift_points_radially(vertices_3d, angs[i])
        
        pts_3D.append(elv_pts) # append points to list
        tris_3D.append(tris) # append triangle 
    
    #####
    # NORMALIZE RADIUS TO FARTHEST POINT FROM ORIGIN
    #####
    
    dists = []
    for i in range(len(pts_3D)):
        for j in range(len(pts_3D[i])):
            dists.append(np.sqrt( ((pts_3D[i][j][0])**2 +
                                   (pts_3D[i][j][1])**2 +
                                   (pts_3D[i][j][2])**2)
                                ))
    
    max_dist = np.max(dists) # get the max radius to normalize by
    
    norm_pts_3D = [] # normalize 3D points
    for i in range(len(pts_3D)):
        norm_pts_3D.append(pts_3D[i]/max_dist)
    
    pts_3D = norm_pts_3D
    
    #####
    # CREATE MESH
    #####
    
    # create mesh of first leaf
    mesh = trimesh.Trimesh(vertices=pts_3D[0], faces=tris_3D[0])
    
    # concatenate the meshes of other leaves
    for i in range(1, len(pts_3D)):
        new_mesh = trimesh.Trimesh(vertices=pts_3D[i], faces=tris_3D[i])
        mesh = trimesh.util.concatenate(mesh, new_mesh)
    
    # Get the vertices (coordinates of each vertex)
    vertices = mesh.vertices
    
    # Get the faces (indices of vertices that form each triangle)
    faces = mesh.faces
    
    #####
    # CALCULATE RAY ORIGINS
    #####
    
    fibo_pts = fibonacci_sphere(samples = n_origins) # calculate points
    fibo_arr = np.array(fibo_pts) # turn tuples into array
    ray_origins = fibo_arr[fibo_arr[:,2]>0]*radius # select points in the positive hemisphere
    
    #####
    # PERFORM RAY CASTING
    #####
    
    # --- Get face centers of mesh ---
    face_centers = mesh.triangles_center  # shape (M, 3)
    n_faces = len(face_centers)
    n_origins = len(ray_origins)
    
    # --- Build ray origins and directions ---
    ray_origins_batch = []
    ray_directions_batch = []
    intended_face_indices = []
    
    for origin_idx, origin in enumerate(ray_origins):
        dirs = face_centers - origin  # (n_faces, 3)
        dirs /= np.linalg.norm(dirs, axis=1, keepdims=True)
        
        ray_origins_batch.append(np.repeat(origin[np.newaxis, :], n_faces, axis=0))
        ray_directions_batch.append(dirs)
        intended_face_indices.append(np.arange(n_faces))  # face indices
    
    # Flatten arrays
    ray_origins_batch = np.vstack(ray_origins_batch)  # shape (N, 3)
    ray_directions_batch = np.vstack(ray_directions_batch)  # shape (N, 3)
    intended_face_indices = np.tile(np.arange(n_faces), n_origins)  # shape (N,)
    
    # --- Subsample rays ---
    N_total = len(ray_origins_batch)
    n_sample = min(n_sample, N_total)
    sample_idx = np.random.choice(N_total, size=n_sample, replace=False)
    
    ray_origins_sub = ray_origins_batch[sample_idx]
    ray_directions_sub = ray_directions_batch[sample_idx]
    intended_faces_sub = intended_face_indices[sample_idx]
    
    # --- Cast rays ---
    locations, ray_index, tri_index = mesh.ray.intersects_location(
        ray_origins_sub, 
        ray_directions_sub, 
        multiple_hits=False
    )
    
    # --- Determine interception correctness ---
    hit_intended = tri_index == intended_faces_sub[ray_index]
    proportion_correct = np.sum(hit_intended) / len(ray_index)
    proportion_incorrect = 1 - proportion_correct
    total_area = mesh.area

    return proportion_correct, proportion_incorrect, self_intersect, total_area, vscaled_lfs

In [ ]:
##################################################
##################################################
##################################################
##################################################
##################################################
# GENERATE PARAMETER FILE
##################################################
##################################################
##################################################
##################################################
##################################################

# save parameter name (without extension)
parameter_name = "nahuatl_parameters_may_11_2025"

parameter_file = pd.DataFrame({

# save parameter name
"parameter_name":parameter_name,

# morphospace and PC values to synthesize leaf shape
"morphospace_file":["nahuatl_morphospace_may_10_2025.pkl"], # PCA model
"PC_val_file":["nahuatl_PCs_may_10_2025.npy"], # empirical PCA values from the PCA

# select two PC axes to sythesize leaves from
"PCa":[0], # index position of first PC (index 0 = PC1)
"PCb":[4], # index position of second PC (index 0 = PC1)

# set density of grid to sample points within leaves
"grid_density":[0.01], # density to sample points on leaves

# select the range of leaf numbers to uniformly sample from
"low_lf_num":[3], # inclusive
"high_lf_num":[10], # up to, not including

# set relative height of first leaf in series
"low_juv_len":[0.2], # inclusive
"high_juv_len":[1], # not including

# set height of last leaf relative to one
"low_adult_len":[0.2], # inclusive
"high_adult_len":[1], # not including

# set maximum scaling values for leaf curvature and leaf curling
"lf_curve":[0.8], # max scaling of leaf curvature (the interior of the leaf)
"lf_curl":[0.8], # max scaling of leaf curl (across length of leaf)

# set maximum displacement at leaf tip (y value)
"max_down_curl":[-1],
"max_up_curl":[1],

# set minimum and maximum limits of leaf angles
"ang_min_limit":[10], # max possible min angle
"ang_max_limit":[90], # max possible angle

# set phyllotaxy angle (in degrees)
# select from options below:
# 0 = [-137.5077640500378546463487,137.5077640500378546463487]
# 1 = [-137.5077640500378546463487,137.5077640500378546463487,90,180]
# 2 = [-137.5077640500378546463487,137.5077640500378546463487,90,180,<random -360 to +360>]
# 3 = [<random -360 to +360>]
"phyllo_ang":[2]

})

# save parameters to file
parameter_file.to_csv(parameter_name+".csv", index=False)

##################################################
##################################################
##################################################
##################################################
##################################################
# CREATE MORPHOLOGICAL MODELS
##################################################
##################################################
##################################################
##################################################
##################################################

# double check parameter name (no file extension) 
parameter_name = "nahuatl_parameters_may_11_2025"

# specify the number of desired models
model_num = 1000

# create folder to save model values
destination_folder_path = "model_vals" 

if not os.path.exists(destination_folder_path): # check if the folder exists
    # create the folder if it doesn't exist
    os.makedirs(destination_folder_path)

for i in range(model_num):

    # GENERATE MODEL VALUES
    # FUNCTION: generate_model_vals()
    parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y, end_x,end_y = generate_model_vals(parameter_name+".csv")
    
    # CREATE MODEL VAL DATAFRAME
    # FUNCTION: get_model_val_df()
    model_val_df = get_model_val_df(parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y, end_x,end_y)

    # SAVE MODEL VALUES
    # file names = "x" + i + parameter space name
    model_val_df.to_csv('./'+destination_folder_path+"/x"+str(i)+"_"+parameter_name+".csv", index=False) 

##################################################
##################################################
##################################################
##################################################
##################################################
# RAY CASTING
##################################################
##################################################
##################################################
##################################################
##################################################

####################
# FETCH MODEL VALUES
####################
directory_path = "./model_vals" # select path of model values
file_names = os.listdir(directory_path) # get file names

for i in range(len(file_names)): # make sure no hidden files
    if file_names[i][0]==".":
        file_names.remove(i)
        
####################
# CREATE FOLDER TO SAVE RESULTS
####################
# create folder to save ray casting results
results_path = "ray_casting_results" 

# create folder to save leaf shape validation
shape_path = "leaf_shape_results" 

if not os.path.exists(results_path): # check if the folder exists
    # create the folder if it doesn't exist
    os.makedirs(results_path)

if not os.path.exists(shape_path): # check if the folder exists
    # create the folder if it doesn't exist
    os.makedirs(shape_path)
    
####################
# RETRIEVE MODEL AND PERFORM RAY CASTING
####################

problem_models = []
for file in file_names: # for each model
    
    print(file) # print file name

    # GET MODEL VALUES FOR THE CURRENT MODEL
    # FUNCTION: get_model_vals()
    parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y, end_x,end_y = get_model_vals(model_val_file_name=directory_path+"/"+file)
    
    # PARAMETERS FOR RAY CASTING
    n_origins = 1000 # set number of sphere points, ray origins
    radius = 2 # set radius of hemisphere of ray origins
    
    # SET HOW MANY RAYS TO SAMPLE
    n_sample = 1000 
    
    # SET HOW MANY LEAF SHAPES IN SERIES TO VALIDATE
    v = 5
    
    # PERFORM RAY CASTING FOR MODELS
    # FUNCTION: ray_casting()
    try:
        proportion_correct, proportion_incorrect, self_intersect, total_area, vscaled_lfs = ray_casting(n_origins,radius,n_sample,v,parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y, end_x,end_y)
        # SAVE RESULTS
        np.save(results_path+'/results_'+file[:-4]+".npy", np.array([proportion_correct, proportion_incorrect, self_intersect, total_area]))
        np.save(shape_path+'/shapes_'+file[:-4]+".npy", vscaled_lfs)
    except Exception as e:
        problem_models.append(file)
        np.savetxt("problem_models.txt", np.array(problem_models, dtype=str), fmt="%s", newline='\n')

        


x256_nahuatl_parameters_may_11_2025.csv
x626_nahuatl_parameters_may_11_2025.csv
x19_nahuatl_parameters_may_11_2025.csv
x415_nahuatl_parameters_may_11_2025.csv
x590_nahuatl_parameters_may_11_2025.csv
x848_nahuatl_parameters_may_11_2025.csv
x900_nahuatl_parameters_may_11_2025.csv
x885_nahuatl_parameters_may_11_2025.csv
x681_nahuatl_parameters_may_11_2025.csv
x73_nahuatl_parameters_may_11_2025.csv
x704_nahuatl_parameters_may_11_2025.csv
x374_nahuatl_parameters_may_11_2025.csv
x822_nahuatl_parameters_may_11_2025.csv
x147_nahuatl_parameters_may_11_2025.csv
x537_nahuatl_parameters_may_11_2025.csv
x315_nahuatl_parameters_may_11_2025.csv
x290_nahuatl_parameters_may_11_2025.csv
x12_nahuatl_parameters_may_11_2025.csv
x765_nahuatl_parameters_may_11_2025.csv
x843_nahuatl_parameters_may_11_2025.csv
x556_nahuatl_parameters_may_11_2025.csv
x126_nahuatl_parameters_may_11_2025.csv
x647_nahuatl_parameters_may_11_2025.csv
x237_nahuatl_parameters_may_11_2025.csv
x78_nahuatl_parameters_may_11_2025.csv
x181

# Problem files

In [21]:
prob_files = np.loadtxt("./problem_models.txt", delimiter=",", dtype=str)

i=0
file = prob_files[i]

model_val_file = "./model_vals/"+file

# GET MODEL VALUES FOR THE CURRENT MODEL
# FUNCTION: get_model_vals()
parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y, end_x,end_y = get_model_vals(model_val_file_name="./model_vals/"+file)

# PARAMETERS FOR RAY CASTING
n_origins = 1000 # set number of sphere points, ray origins
radius = 2 # set radius of hemisphere of ray origins

# SET HOW MANY RAYS TO SAMPLE
n_sample = 1000 

# SET HOW MANY LEAF SHAPES IN SERIES TO VALIDATE
v = 5

# PERFORM RAY-CASTING, SEE ERROR
proportion_correct, proportion_incorrect, self_intersect, total_area, vscaled_lfs = ray_casting(n_origins,radius,n_sample,v,parameter_name, morphospace_file, PC_val_file, PCa, PCb, grid_density, low_lf_num, high_lf_num, low_juv_len, high_juv_len, low_adult_len, high_adult_len, lf_curve, lf_curl, max_down_curl, max_up_curl, ang_min_limit, ang_max_limit, phyllo_ang,n,juv_len,node_max,adult_len,leaf_curve,leaf_curl,leaf_tip,leaf_mid,angle_min,angle_max,start_x,start_y, end_x,end_y)
